In [ ]:
#TODO
#1. Add the 64*64 outputs to one image.
#2. For every pixel, convert it into the most probable one.Maybe depend on neighbours??
# Could be multiple overlaps, find a way to handle that.
#3. Convert every pixel to correct polygon json format

In [ ]:
import numpy as np

In [2]:
def post_process_optimized(outputs, gamma=0.5):
    empty_img = np.zeros((1024, 1024, 5))
    processed_img = np.zeros((1024, 1024), dtype=int)
    
    for x, y, probs in outputs:
        empty_img[x:x+64, y:y+64] += np.array(probs)
    
    padded = np.pad(empty_img, ((1,1), (1,1), (0,0)))
    neighbors = gamma * (padded[:-2, 1:-1] + padded[2:, 1:-1] + 
                        padded[1:-1, :-2] + padded[1:-1, 2:])
    
    processed_img = np.argmax(empty_img + neighbors, axis=2)
    return processed_img

def test_post_process_optimized():
    positions = np.mgrid[0:1024:64, 0:1024:64].reshape(2, -1).T
    
    probs = np.random.dirichlet(np.ones(5)*2, size=positions.shape[0])
    probs[:,0] = np.clip(probs[:,0], 0.3, 1.0)
    probs = probs / probs.sum(axis=1, keepdims=True)
    
    outputs = [(x, y, p.tolist()) for (x, y), p in zip(positions, probs)]
    return post_process_optimized(outputs)

## torch and cuda version

In [ ]:
import torch

def post_process_torch(outputs, gamma=0.5):
    device = torch.device("cuda")  # Assuming model outputs are on CUDA
    empty_img = torch.zeros((1024, 1024, 5), device=device)
    
    # Convert outputs to tensor once
    coords = torch.tensor([[x,y] for x,y,_ in outputs], device=device)
    probs = torch.tensor([p for _,_,p in outputs], device=device)
    
    # Batch assignment
    for coord, prob in zip(coords, probs):
        empty_img[coord[0]:coord[0]+64, coord[1]:coord[1]+64] += prob
    
    padded = torch.nn.functional.pad(empty_img, (0,0,1,1,1,1))
    neighbors = gamma * (padded[:-2, 1:-1] + padded[2:, 1:-1] + 
                        padded[1:-1, :-2] + padded[1:-1, 2:])
    
    return torch.argmax(empty_img + neighbors, dim=2)  # Keep on GPU if needed